In [1]:
# Imports

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import os
import ray
from ray.rllib.algorithms.ppo import PPOConfig
from ray import tune

from src.parsers import HMParser
from src.resources import BinaryResource, Generator, Load, Storage, Vehicle
from src.algorithms.rl import EVMultiAgent_v0

2023-11-26 20:53:47,040	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [2]:
# Data parsing

data = HMParser(file_path='data/EC_V4.xlsx', ec_id=1)
data.parse()

In [3]:
# Create EVs

evs = []
for ev in np.arange(data.vehicle['e_capacity_max'].shape[0]):
    new_ev = Vehicle(name='ev_{:02d}'.format(ev + 1),
                     value=data.vehicle['min_technical_soc'][ev] * data.vehicle['e_capacity_max'][ev],
                     lower_bound=np.ones(data.vehicle['schedule_charge'][ev].shape) * \
                                 data.vehicle['min_technical_soc'][ev] * data.vehicle['e_capacity_max'][ev],
                     upper_bound=np.ones(data.vehicle['schedule_charge'][ev].shape) * \
                                 data.vehicle['e_capacity_max'][ev],
                     cost = np.zeros(data.vehicle['schedule_charge'][ev].shape),
                     cost_discharge=np.ones(data.vehicle['schedule_charge'][ev].shape) * \
                                    data.vehicle['discharge_price'][ev, 0],
                     cost_charge=np.ones(data.vehicle['schedule_charge'][ev].shape) * \
                                 data.vehicle['charge_price'][ev, 0],
                     capacity_max=data.vehicle['e_capacity_max'][ev],
                     initial_charge=data.vehicle['min_technical_soc'][ev] * data.vehicle['e_capacity_max'][ev],
                     min_charge=data.vehicle['min_technical_soc'][ev] * data.vehicle['e_capacity_max'][ev],
                     discharge_efficiency=data.vehicle['charge_efficiency'][ev],
                     charge_efficiency=data.vehicle['charge_efficiency'][ev],
                     schedule_connected=data.vehicle['schedule'][ev],
                     schedule_discharge=data.vehicle['schedule_discharge'][ev],
                     schedule_charge=data.vehicle['schedule_charge'][ev],
                     schedule_requirement_soc=data.vehicle['schedule_departure_soc'][ev],
                     schedule_arrival_soc=data.vehicle['schedule_arrival_soc'][ev]
                     )
    evs.append(new_ev)

In [4]:
# Create and test environment to check if everything is working

temp_env = EVMultiAgent_v0(resources=evs,
                           penalty_action_coefficient=2.0,
                           penalty_not_charged=100.0,
                           energy_price=data.peers['buy_price'][0, :])
temp_env.reset()
terminations = truncations = {a: False for a in temp_env.agents}
terminations['__all__'] = False
truncations['__all__'] = False
while not terminations['__all__'] and not truncations['__all__']:
    actions = temp_env.action_space.sample()
    observations, rewards, terminations, truncations, infos = temp_env.step(actions)
    # print('Observation: {}'.format(observations))
    # print('Observation keys: {}'.format(observations.keys()))
    # print('Reward: {}'.format(rewards))
    # print('Infos: {}\n'.format(infos))
print('Terminated: {}'.format(terminations['__all__']))

Terminated: True
